# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import re
import os
import pickle

# Data reading and cleaning

In [3]:
DATA_DIR = 'data'
FILE_NAME = 'train_1'
CSV_PATH = os.path.join(DATA_DIR, FILE_NAME + '.csv')
PKL_PATH = os.path.join(DATA_DIR, FILE_NAME + '.pkl')

def load_data() -> pd.DataFrame:
    '''
    Loads data from path. If there is a cached version loads it instead.
    '''
    if os.path.exists(PKL_PATH):
        print('Loading pickle...')
        df = pd.read_pickle(PKL_PATH)
        print('Done!')
        return df
    else:
        print('Loading csv...')
        df = pd.read_csv(CSV_PATH)
        df.to_pickle(PKL_PATH)
        print('Done!')
        return df

In [4]:
def read_all() -> pd.DataFrame:
    '''
    Loads data, sets index for the df and makes columns a date type.
    Also pickles for speed increase
    '''
    
    treated_pkl = os.path.join(DATA_DIR, 'treated.pkl')
    if os.path.exists(treated_pkl):
        df = pd.read_pickle(treated_pkl)
    else:
        df = load_data()
        df.set_index('Page', inplace=True)
        df.sort_index(inplace=True)
        df.columns = df.columns.astype('M8[D]')
        print('Pickling treated data...')
        df.to_pickle(treated_pkl)
        print('Done!')
    return df

In [5]:
def read_interval(start, end) -> pd.DataFrame:
    '''
    Returns dataframe within specified values: ts[start:end]
    '''
    df = read_all()
    if start and end:
        return df.loc[:, start:end]
    elif end:
        return df.loc[:, :end]
    else:
        return df

In [6]:
df = read_interval(None, None)

In [7]:
df.head()

,2015-07-01 00:00:00,2015-07-02 00:00:00,2015-07-03 00:00:00,2015-07-04 00:00:00,2015-07-05 00:00:00,2015-07-06 00:00:00,2015-07-07 00:00:00,2015-07-08 00:00:00,2015-07-09 00:00:00,2015-07-10 00:00:00,...,2016-12-22 00:00:00,2016-12-23 00:00:00,2016-12-24 00:00:00,2016-12-25 00:00:00,2016-12-26 00:00:00,2016-12-27 00:00:00,2016-12-28 00:00:00,2016-12-29 00:00:00,2016-12-30 00:00:00,2016-12-31 00:00:00
Page,,,,,,,,,,,,,,,,,,,,,
!vote_en.wikipedia.org_all-access_all-agents,3.0,4.0,7.0,4.0,4.0,2.0,3.0,7.0,2.0,NaN,...,3.0,1.0,6.0,3.0,1.0,1.0,4.0,3.0,1.0,1.0
!vote_en.wikipedia.org_all-access_spider,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,2.0,NaN,...,2.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0
!vote_en.wikipedia.org_desktop_all-agents,3.0,4.0,7.0,4.0,4.0,2.0,3.0,7.0,2.0,NaN,...,3.0,1.0,6.0,3.0,1.0,1.0,3.0,3.0,1.0,0.0
"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4923.0,5074.0,4149.0,4209.0,3783.0,3994.0,3910.0,4006.0,3841.0,3517.0
"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.0,12.0,63.0,15.0,38.0,14.0,59.0,40.0,35.0,45.0


In [8]:
df.shape

(145063, 550)

16 mins with dataframe access <br>
6 segs with numpy arrays <br>
289ms with numba <br>

In [9]:
%%time
import time
import numba

@numba.jit(nopython=True)
def calculate_start_end(data: np.ndarray):
    '''
    Calculates start and end of series.
    Start = first non nan value
    End = last non nan value
    '''
    rows, columns = data.shape[0], data.shape[1]
    
    # array where idx will be stored. if none is found, defaults to -1
    start_idx = np.full(rows, -1, dtype=np.int32)
    end_idx = np.full(rows, -1, dtype=np.int32)
    
    for page in range(rows):
        #start idx
        for day in range(columns):
            # if nan 1, get index for 1
            if not np.isnan(data[page, day]) and data[page, day] > 0:
                start_idx[page] = day
                break
        for day in range(columns -1, -1, -1):
            if not np.isnan(data[page, day]) and data[page, day] > 0:
                end_idx[page] = day
                break
        
    return start_idx, end_idx

start, end = calculate_start_end(df.values)



CPU times: user 587 ms, sys: 37.3 ms, total: 624 ms
Wall time: 3.65 s


In [10]:
def clean_data(threshold, start=None, end=None):
    '''
    Loads data, setting Page as index, and columns as datetime dtypes.
    Removes series that don't comply to minimum threshold of nan to value ratio
    Returns normalized series (log1p), indexes of previously nan values, start and end indexes
    '''
    df = read_interval(start, end)
    start, end = calculate_start_end(df.values)
    bool_mask = ~(((end - start) / df.shape[1]) < threshold)
    df = df[bool_mask]
    
    nan_values = pd.isnull(df)
    return np.log1p(df.fillna(0)), nan_values, start, end

In [86]:
df, nans, starts, ends = clean_data(0.3)

# Date Range Preprocessing

In [20]:
start_date, end_date = df.columns[0], df.columns[-1]

In [21]:
forecast_days = 60
prediction_window = end_date + pd.Timedelta(forecast_days, unit='D')
end_date, prediction_window

(Timestamp('2016-12-31 00:00:00'), Timestamp('2017-03-01 00:00:00'))

# Autocorrelation

$r^{k} = \frac{\sum_{i=1}^{N-k}(Y_{i} - \bar{Y})(Y_{i+k} - \bar{Y})}{\sum_{i=1}^{N}(Y_{i} - \bar{Y})^{2}}$

In [97]:
@numba.jit(nopython=True)
def autocorrelation(series: np.ndarray, days):
    '''
    Calculates autocorrelation for series according to Box Jenkins
    '''
    ts = series[days:]
    ts_lag = series[:-days]
    dts = ts - np.mean(ts)
   
    dts_lag = ts_lag - np.mean(ts_lag)
    
    
    #denominator = np.sum(np.square(dts))
    denominator = np.sqrt(np.sum(dts * dts)) + np.sqrt(np.sum(dts_lag * dts_lag))
    nominator = np.sum(dts * dts_lag)
    if denominator == 0 or np.isnan(denominator):
        autocorr = 0
    else:
        autocorr = nominator / denominator
    return autocorr



170 ns to 110 ns 

In [98]:
%%timeit
autocorrelation(df.iloc[0].values, 365)


The slowest run took 3743.93 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 126 µs per loop


In [115]:
np.sum(ends == -1)

752

In [127]:
@numba.jit(nopython=True)
def batch_autocorrelation(tseries, lag, starts, ends, threshold):
    '''
    Calculates autocorrelation for lots of series.
    Checks if we are calculating a meaningful result for autocorrelation
    If the length of the series is less than the threshold (len/lag),
    autocorrelation is none.
    '''
    rows, columns = tseries.shape[0], tseries.shape[1]
    corr = np.full(rows, np.nan, dtype=np.float64)
    
    for i in range(rows):
        # check if series complies to threshold
        start = starts[i]
        end = min(ends[i], columns)
        ratio = (end - start) / lag
        if ratio > threshold:
            # calculate autocorrelation
            entry = tseries[i]
            corr[i] = autocorrelation(entry[start:end], lag)
        else:
            # autocorr is nan
            continue
            
    return corr

In [152]:
corr = batch_autocorrelation(df.values, 365, starts, ends, 1.5)
(corr.shape[0] - np.sum(np.isnan(corr)) ) / corr.shape[0]

0.84636133986041928

In [147]:
def standard_scale(arr: np.ndarray):
    '''
    Normalize data (x - mean)/std
    '''
    return (arr - arr.mean()) / np.std(arr)

In [166]:
def get_autocorr(tseries: np.ndarray, lags: list, starts, ends, threshold, normalize=True):
    '''
    Gets all autocorrelations for each specified lag.
    '''
    
    corr = [batch_autocorrelation(tseries, lag, starts, ends, threshold)
           for lag in lags]
        
    for i in range(len(lags)):
        ratio = (corr[i].shape[0] - np.sum(np.isnan(corr[i])) ) / corr[i].shape[0]
        nan_percent = 1 - ratio
        print("For lag: %i nan percent is %.3f" % (lags[i], nan_percent))
        
    if normalize:
        corr = [standard_scale(np.nan_to_num(batch)) for batch in corr]
    
    return corr
    

In [167]:
lags = [int(round(365/4)), 365] #yearly and quarterly
get_autocorr(df.values, lags, starts, ends, 1.5, normalize=True)

For lag: 91 nan percent is 0.029
For lag: 365 nan percent is 0.154


[array([-0.35653493, -0.29128775, -0.29947322, ..., -0.00597079,
        -0.33125713, -0.32009653]),
 array([-0.15618991, -0.35558243, -0.13367393, ..., -0.26995498,
        -0.49244841, -0.50215723])]

# Extracting features from URL

In [ ]:
pat = re.compile(
    '(.+)_([a-z]{2}\.)?((?:wikipedia\.org)|(?:commons\.wikimedia\.org)|(?:www\.wikimedia\.org)|(?:www\.mediawiki\.org))_([a-z-]+?)_([a-z-]+)$'
)

def extract_from_url(urls):
    '''
    receives pandas dataframe column or series
    returns a pandas dataframe with all the extracted features
    '''
    
    if isinstance(urls, pd.Series):
         urls = urls.values
    
    accesses = np.full_like(urls, np.NaN)
    agents = np.full_like(urls, np.NaN)
    sites = np.full_like(urls, np.NaN)
    countries = np.full_like(urls, np.NaN)
    titles = np.full_like(urls, np.nan)
    
    for i in range(len(urls)):
        url = urls[i]
        match = pat.fullmatch(url)
        assert match, "regex pattern matching failed %s" % url
        
        titles[i] = match.group(1)
        
        country = match.group(2)
        if country:
            countries[i] = country[:-1]
        else:
            countries[i] = 'na'
            
        sites[i] = match.group(3)
        
        agents[i] = match.group(4)
        accesses[i] = match.group(5)
        
    return pd.DataFrame({
        'page': urls,
        'title': titles,
        'agent': agents,
        'access': accesses,
        'site': sites,
        'country': countries,
    })